In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tag import pos_tag
import matplotlib.pyplot as plt
import string
import multiprocessing
import os
import copy
import re
import sklearn
import seaborn as sns
from collections import Counter
import wordcloud
%matplotlib inline
from subprocess import check_output

## Read dataset
read all the data and convert them into pandas dataframes

In [ ]:
import json
from pprint import pprint

cwd = os.getcwd()
# print(cwd)
frames = []
for file in os.listdir('lyrics_50_18/')[1500:]:
    date = file.split('.')[0].split('_')[-1]
    with open(cwd + '/lyrics_50_18/' + file,'r') as f:
        df = pd.DataFrame(json.load(f)).dropna()
        df['time'] = pd.Timestamp(date)
        frames.append(df)
raw_data = pd.concat(frames).reset_index()  #.drop_duplicates(['artist', 'title']) 

In [ ]:
raw_data

## Data preprocessing
1. str to lower
2. remove stop words
3. remove non-lyric content (eg: [Chorus: David McRae])
4. remove punctuations

In [ ]:
import copy
billboard = copy.deepcopy(raw_data)
stop = stopwords.words('english')

def preprocess(lyric):
    lyric = lyric.lower()
    lyric = re.sub(r'\[.*?\]', '', lyric)  # remove [*] pattern
    lyric = lyric.replace("'s",'')
    lyric = lyric.replace("'ve",'')
    lyric = lyric.replace("'",'')    # ' must be ignored
    lyric = lyric.replace("-",' ')
    
    translator = str.maketrans(string.punctuation,' '*len(string.punctuation))
    lyric = lyric.translate(translator)
    tokens = nltk.word_tokenize(lyric)
    
    return tokens


In [ ]:
billboard['lyrics'] = billboard['lyrics'].apply(lambda x: preprocess(x))
billboard['lyrics'] = billboard['lyrics'].apply(lambda x:[w for w in x if w not in stop])
billboard.head(10)

In [ ]:
billboard['vocab_cnt'] = billboard['lyrics'].str.len()
billboard.head(10)

In [ ]:
total_year_cnt = billboard.groupby(billboard.time.dt.year)['vocab_cnt'].sum()
plt.figure(figsize=(12, 9))
plt.title("Lyrics Vocabulary Count by Year", fontsize=20)
total_year_cnt.plot(kind='line', color='Red')

In [ ]:
def get_NN_pos_tag(lyric):
    return len([w for w, pos in pos_tag(lyric) if pos == 'NN'])

billboard['nouns_cnt'] = billboard['lyrics'].apply(lambda x: get_NN_pos_tag(x))
billboard.head(10)

In [ ]:
nouns_cnt = billboard.groupby(billboard.time.dt.year)['nouns_cnt'].sum()
plt.figure(figsize=(12, 9))
plt.title("Nouns Count by Year in Lyric", fontsize=20)
nouns_cnt.plot(kind='line', color='Blue')

In [ ]:
def get_Adj_pos_tag(lyric):
    return len([w for w, pos in pos_tag(lyric) if pos == 'JJR'])

billboard['adjs_cnt'] = billboard['lyrics'].apply(lambda x: get_Adj_pos_tag(x))
billboard.head(10)

In [ ]:
adjs_cnt = billboard.groupby(billboard.time.dt.year)['adjs_cnt'].sum()
plt.figure(figsize=(12, 9))
plt.title("Adjs Count by Year in Lyric", fontsize=20)
adjs_cnt.plot(kind='line', color='Green')

In [ ]:
artist_cnt = billboard.artist.value_counts()[:25]
artist_cnt

In [ ]:
plt.figure(figsize=(13,9))
plt.title("Maximum Vocabulary Count By Artist",fontsize=20)
vocab_cnt_by_artist = billboard.groupby('artist').agg({'vocab_cnt': sum}) \
                        .sort_values(['vocab_cnt'], ascending=False).vocab_cnt[:30]
vocab_cnt_by_artist.plot('bar',color='teal')

In [ ]:
plt.figure(figsize=(12,5))
plt.title("Max Repeated Words In Song",fontsize=20)
words = pd.Series(billboard['lyrics'].sum()).value_counts()
words.plot(color='orange')

## 